# Libraries

In [1]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
import pandas as pd
import psycopg2
import os

In [2]:
load_dotenv()

True

# Database

In [3]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DB = os.getenv('POSTGRES_DB')

In [4]:
engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

In [5]:
Session = sessionmaker(bind=engine)
session = Session()

# Data

In [6]:
sql_query = text("""
WITH global_climate_and_forest_fire AS (
    SELECT * FROM global_climate 
    JOIN forest_fire USING(latitude, longitude, \"date\")
), 
global_climate_and_forest_fire_and_ndvi AS (
    SELECT * FROM global_climate_and_forest_fire 
    JOIN ndvi USING(latitude, longitude, \"date\")
), land_cover_data AS (
    SELECT lc.latitude, lc.longitude, lc."year", 
    lcl."General class", lcl."class", lcl."Sub-class"
    FROM land_cover lc JOIN land_cover_legend lcl
    ON lc.land_cover = lcl."Map value"
),
population_density_and_land_cover AS (
    SELECT * FROM population_density 
    JOIN land_cover_data USING(latitude, longitude, \"year\")
), 
all_data AS (
    SELECT DISTINCT * FROM population_density_and_land_cover pd_lc
    JOIN global_climate_and_forest_fire_and_ndvi gc_ff_nd
    USING(latitude, longitude) 
    WHERE pd_lc.\"year\" = EXTRACT(YEAR FROM gc_ff_nd.\"date\")
)

SELECT * FROM all_data;
""")
data = session.execute(sql_query)

In [7]:
rows = data.fetchall()
df = pd.DataFrame(rows, columns=data.keys())

In [8]:
columns = {
    "latitude": "latitude",
    "longitude": "longitude",
    "population_density": "population_density",
    "General class": "land_cover_type",
    "class": "land_cover_subtype",
    "Sub-class": "vegetation_percent",
    "date": "date",
    "ws": "win_speed",
    "vpd": "vapor_pressure_deficit",
    "vap": "vapor_pressure",
    "tmin": "minimum_temperature",
    "tmax": "maximum_temperature",
    "swe": "snow_water_equivalent",
    "srad": "surface_shortwave_radiation",
    "soil": "soil_moisture",
    "q": "runoff",
    "ppt": "precipitation_accumulation",
    "pet": "Reference_evapotranspiration",
    "def": "climate_water_deficit",
    "aet": "actual_Evapotranspiration",
    "PDSI": "palmer_drought_severity_index",
    "brightness": "brightness_temperature",
    "bright_t31": "brightness_temperature_31",
    "scan": "scan_fire_size",
    "track": "track_fire_size",
    "confidence": "confidence",
    "frp": "fire_radiative_power",
    "daynight": "daynight",
    "type": "fire_type",
    "n_pixels": "n_pixels_ndvi",
    "vim": "ndvi",
    "vim_avg": "ndvi_long_term_average",
    "viq": "ndvi_anomaly_percent"
}

In [9]:
df_final = df.rename(columns=columns)[columns.values()]
df_final.head(5)

,latitude,longitude,population_density,land_cover_type,land_cover_subtype,vegetation_percent,date,win_speed,vapor_pressure_deficit,vapor_pressure,...,scan_fire_size,track_fire_size,confidence,fire_radiative_power,daynight,fire_type,n_pixels_ndvi,ndvi,ndvi_long_term_average,ndvi_anomaly_percent
0,-4.223234,-69.970612,154.694519,Open surface water,Open surface water,90-100% of year,2017-10-14,1.47,0.82,3.060,...,0.38,0.36,l,3.72,D,offshore,198.0,0.8564,0.8548,100.1807
1,-4.220858,-69.962410,199.979401,Built-up,Built-up,Built-up,2020-08-19,1.20,1.00,2.898,...,0.38,0.36,n,3.03,D,offshore,198.0,0.8588,0.8557,100.3475
2,-4.216577,-69.969467,194.736084,Wetland,Dense short vegetation,100% short vegetation cover,2013-09-22,1.15,0.85,2.953,...,0.41,0.37,n,2.84,D,offshore,198.0,0.8528,0.8541,99.8581
3,-4.216006,-69.965797,151.117371,Wetland,Dense short vegetation,100% short vegetation cover,2013-09-22,1.15,0.85,2.953,...,0.41,0.37,n,3.83,D,offshore,198.0,0.8528,0.8541,99.8581
4,-4.213908,-69.962715,147.899872,Wetland,Dense short vegetation,100% short vegetation cover,2016-10-26,1.47,0.89,3.136,...,0.38,0.43,n,3.48,D,offshore,198.0,0.8572,0.8555,100.1846


# Model

In [48]:
import math

latitudes, longitudes = df_final['latitude'], df_final['longitude']
lat_min, lon_min = latitudes.min(), longitudes.min()
lat_max, lon_max = latitudes.max(), longitudes.max()

lat_min, lon_min = math.floor(lat_min), math.floor(lon_min)
lat_max, lon_max = math.ceil(lat_max), math.floor(lon_max)

url = "https://opendap.cr.usgs.gov/opendap/hyrax/DP109/SRTM/SRTMGL1N.003/2000.02.11/S05W049.SRTMGL1N.num.zip"
uri = "https://opendap.cr.usgs.gov/opendap/hyrax/DP109/SRTM/SRTMGL1N.003/2000.02.11/S05W061.SRTMGL1N.num.zip"

print(f"S0{abs(lat_min)}W0{abs(lon_min)} - N{lat_max}W0{abs(lon_max)}")
for i in range(lat_max - lat_min + 1):
    for j in range(lon_max - lon_min + 1):
        r = "S0" if lat_min + i < 0 else "N0" if lat_min + i < 10 else "N"
        polygon = f"{r}{abs(lat_min + i)}W0{abs(lon_min + j)}"
        url = "https://opendap.cr.usgs.gov/opendap/hyrax/DP109/SRTM/SRTMGL1N.003/2000.02.11/{polygon}.SRTMGL1N.num.zip"

S05W082 - N13W067
S05W082
S05W081
S05W080
S05W079
S05W078
S05W077
S05W076
S05W075
S05W074
S05W073
S05W072
S05W071
S05W070
S05W069
S05W068
S05W067
S04W082
S04W081
S04W080
S04W079
S04W078
S04W077
S04W076
S04W075
S04W074
S04W073
S04W072
S04W071
S04W070
S04W069
S04W068
S04W067
S03W082
S03W081
S03W080
S03W079
S03W078
S03W077
S03W076
S03W075
S03W074
S03W073
S03W072
S03W071
S03W070
S03W069
S03W068
S03W067
S02W082
S02W081
S02W080
S02W079
S02W078
S02W077
S02W076
S02W075
S02W074
S02W073
S02W072
S02W071
S02W070
S02W069
S02W068
S02W067
S01W082
S01W081
S01W080
S01W079
S01W078
S01W077
S01W076
S01W075
S01W074
S01W073
S01W072
S01W071
S01W070
S01W069
S01W068
S01W067
N00W082
N00W081
N00W080
N00W079
N00W078
N00W077
N00W076
N00W075
N00W074
N00W073
N00W072
N00W071
N00W070
N00W069
N00W068
N00W067
N01W082
N01W081
N01W080
N01W079
N01W078
N01W077
N01W076
N01W075
N01W074
N01W073
N01W072
N01W071
N01W070
N01W069
N01W068
N01W067
N02W082
N02W081
N02W080
N02W079
N02W078
N02W077
N02W076
N02W075
N02W074
N02W073
N02W07

# Close connection

In [12]:
session.close()
engine.dispose()